In [56]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from pgmpy.estimators import K2Score
from rpy2.robjects import conversion, default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import rpy2.robjects as ro
from pgmpy.estimators import IVEstimator
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import BayesianEstimator as BE
from pgmpy.estimators import BDeuScore
from pgmpy.estimators import BicScore
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score
pandas2ri.activate()
from sklearn.metrics import f1_score

### 结构学习和参数学习改为R
### 新建了图表数据

In [57]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix

def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix.loc[index,col]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list  
def print_dag(dag,all_node=False):
    if not all_node:
        if dag.has_node("label"):
            for edges in dag.get_ancestral_graph("label").edges:
                print(edges[0],"->",edges[1],";")
        else:
            print("label have not ancestor")
            for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    else:
         for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    return 
            
def model_auc(prob,y_true):
    y_prob = []
    num = 0
    for i in range(len(prob)):
        y_prob.append(prob.iloc[i,y_true[i]])
    if y_predic[i]==y_true[i]:
        num+=1
    fpr, tpr, thresholds = roc_curve(y_true,y_predic)
    
    return auc(fpr, tpr)
def to_strength_list(edges_strength_martix):
    str_list = []
    for col in edges_strength_martix.columns:
        for idx in edges_strength_matrix.columns:
            #不等于才加
            if edges_strength_matrix[col][idx]!=0:
                str_list.append([idx,col,edges_strength_matrix[col][idx]])
    return str_list

#根据互信息的分数计算黑名单
def caculate_hos_bl(data,bl):    
    mi_bl = []
    for col in data.columns:
        for index in data.columns:
            mi_score = mutual_info_score(data.loc[:,index],data.loc[:,col])
            if col==index:
                continue
            ##互信息分数，如果两个特征的互信息分数小雨0.02的话就将它加入黑名单
            if mi_score<0.01:
                mi_bl.append([index,col])
    h_bl=pd.DataFrame(mi_bl,columns=["from","to"])
    h_bl=pd.concat([h_bl,bl])
   
    return h_bl
def cross_validation(ed,DB):
    
    #cid客户端号，寻找相应的训练数据
    auc_score = []
    recall = []
    acc = []
    f1 = []
    for i in range(len(DB)): 
        
        model_i= BayesianNetwork(ed.edges)
        #创建边
        data = DB[i]
        
        f_1 = model_i.nodes
        
        f_2 = data.columns
        #所有mode有的节点
        drop_name = []
        for j in f_2:
            if not j in f_1:
                drop_name.append(j)        
        data = data.drop(columns = drop_name,axis = 1)
        
        
        #创建离散变量说明
        this_dict = []

        for key,value in zip(state_names.keys(),state_names.values()):
            
            if key in f_1:

                this_dict.append((key,value))
        this_dict = dict(this_dict)
         #训练模型
        
        
        train_data,test_data = train_test_split(data,test_size=0.3,random_state=2)
        #得到y-true
        y_true = test_data.iloc[:,0]
        #测试集
        model_i.fit(train_data,estimator=BE,complete_samples_only=True,state_names=this_dict)
        
        #丢弃label获得
        
        test_data.drop(columns="label",axis =1,inplace=True)
      
        #将预测的值从df转换为list形式
        
        #在predict的时候不知道什么会影响结果
        y_pred = list(model_i.predict(test_data).iloc[:,0])
       
       #模型预测
        acc.append(accuracy_score(y_true,y_pred))
        recall.append(recall_score(y_true, y_pred))
        auc_score.append(roc_auc_score(y_true,y_pred))
        f1.append(f1_score(y_true,y_pred))
        
    return acc,recall,auc_score,f1
def dag_score(dag,DB):
    score =  []
    
    for data in DB:
        #判断在每个网路之间分数
        
#                 k2 = K2Score(data)
#         score.append(k2.score(dag))
        bic = BicScore(data)
        score.append(bic.score(dag))
    score = np.array(score)
    return score
def net_score(ed,data,hos_id):
    

        
  
    ##转换为R字符串
    r_dag = py_to_rdag(ed)
    ##预测
    r_y = ro.r['predict_label'](r_dag,data,hos_id)-1
    for i in range(len(r_y)):
        if r_y[i]<0:
            r_y[i]=0
        elif r_y[i]>1:
            r_y[i]=1

    split_size = int(len(r_y)*0.5) 
    
    y_pred = r_y[:split_size]
    
    y_true = r_y[split_size:]
    
    acc = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    
        
    return acc,recall,auc,f1
def py_to_rdag(dag):
    rdag_str = ""
    for node in dag.nodes():
        tmp_str = "["+node
        if dag.get_parents(node)!=[]:
            tmp_str = tmp_str+"|"
            index = len
            for i in range(len(dag.get_parents(node))):
                if i == len(dag.get_parents(node))-1:
                    tmp_str+=dag.get_parents(node)[i]
                else:
                    tmp_str+=dag.get_parents(node)[i]+":"


        tmp_str+="]"
        rdag_str += tmp_str
    return rdag_str
    

In [69]:
ro.r("""
    data_tofactor<-function(data){

        datacols = names(data)
        
        for (i in 1:ncol(data)) {
        
            data[,datacols[i]] <- factor(data[,datacols[i]])
        }
    return(data)
    }
""")

#R结构
ro.r("""
        
        r_sturct_study<-function(self_r_data,bl,hos_id){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = "k2",blacklist = bl)
            
            arcs = arc.strength(dag,train_data)
            
            return(arcs)
        }
""")

#R结构
ro.r("""
        
        r_sturct_study_wl<-function(self_r_data,bl,wl,hos_id){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = "k2",whitelist=wl,blacklist = bl)
            
            arcs = arc.strength(dag,train_data)
            
            return(arcs)
        }
""")

#定义个一个用来预测的R语言的函数
ro.r("""
    predict_label<-function(r_dag,self_r_data,hos_id){
        library(bnlearn)
        
        model = model2network(r_dag)
        
        self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
        train_size = ceiling(length(self_r_data[,1])*0.7)
            
   
        
        train_data = self_r_data[1:train_size,nodes(model)]
        
        test_data = self_r_data[(train_size+1):nrow(self_r_data),nodes(model)]
    
        training = bn.fit(model,train_data)
        
        predicted = predict(training, node = "label", data = test_data)
        
        y_true = test_data[,"label"]
        
        out <- c(predicted ,y_true)
        
        return(out)
    }
""")


<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x0000018C8570BBC0> [RTYPES.CLOSXP]
R classes: ('function',)

In [70]:
class client():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid):
        #black 用R语言的存储方式存储
        self.score = {}
        self.cid = cid
        self.data=data
        #初始化创建一个空的图
        self.model = DAG()
        
        feature = data.columns
        
        self.edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        #创建记录评分的字典
        self.score = {
            "acc":[],
            "recall":[],
            "auc":[],
            "f1":[]}
        self.dag_score=0
    def bl_update(self,bl):
        #在实例化客户端之后，根据自己数据的互信息值创建黑名单
        self.black_list = caculate_hos_bl(self.data,bl)
    def get_model(self):
        #获得本地客户端网络结构
        return self.model
    def fit(self):
        #=
        data,t_data = train_test_split(self.data,test_size=0.3,random_state=2)
        self_r_data = ro.pandas2ri.py2rpy(data)
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
        
        
        arc_strength = ro.r['r_sturct_study'](DB,self_r_black_list,self.cid)
        arc_strength = ro.pandas2ri.rpy2py_dataframe(arc_strength)
        self.arc_strength =  arc_strength.round(4)
        #创建新的模型，并添加结果
        self.model = DAG()
        print("客户端",self.cid+1,"开始fit")
        for i in range(len(arc_strength["from"])):
            
            #汇集频数
            self.model.add_edge(arc_strength["from"][i],arc_strength["to"][i])
          
            
#            #汇集强度转换为矩阵
            self.edges_strength.loc[[arc_strength["from"][i]],[arc_strength["to"][i]]]=arc_strength["strength"][i]
        self.edges_strength=self.edges_strength.round(4)
        #计算网路得分
        bic = BicScore(self.data)
        self.dag_score=bic.score(self.model)
        return 1
    
    def update_model(self,wl):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
    
        new_dag = DAG()
        
        wl_r =ro.conversion.py2rpy(wl)
        #客户端将自身的黑名单转换为R数据模式传进函数里
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
                                                 
        
        #调用R函数，并接收返回值
        
        arc_strength_r = ro.r['r_sturct_study_wl'](DB,self_r_black_list,wl_r,self.cid)
        
        self.arc_strength=ro.pandas2ri.rpy2py_dataframe(arc_strength_r)
        
        print("客户端",self.cid,"开始update")
        #重新生成一个副本，比较那个性能较高
        for i in range(len(self.arc_strength["from"])):
            #转换自己的网络
            new_dag.add_edge(self.arc_strength["from"][i],self.arc_strength["to"][i])
            #生成新的强度列表
            self.edges_strength.loc[[self.arc_strength["from"][i]],[self.arc_strength["to"][i]]]=self.arc_strength["strength"][i]
        
        
        self.edges_strength=self.edges_strength.round(4)
        
        
        ##目前用AUC计算网络的分数
        
        new_score = net_score(new_dag,DB,self.cid)
        
        old_score = net_score(self.model,DB,self.cid)
        
        
        
        if new_score[2]>old_score[2]:
            self.model = new_dag
        else:
            new_score = old_score
            
        print(new_score)
        
        self.score["acc"].append(new_score[0])
        self.score["recall"].append(new_score[1])
        self.score["auc"].append(new_score[2])
        self.score["f1"].append(new_score[3])
        
    
        bic = BicScore(self.data)
        
        self.dag_score=bic.score(self.model)
        
    def client_score():
        ##获得当前本网络的性能
        r_dag = py_to_rdag(self.model)
        
        r_y = ro.r['predict_label'](r_dag,self_r_data,hos_id)
        split_size = len(r_y)*0.5
        y_pred = r_y[:split_size]
        y_true = r_y[split_size:]
        
        
        
class sever():
    def __init__(self):
        pass
        
    def Fd_caculate(self,edges_strength_matrix,round_num):
        #聚合方法
        #聚合的方式改为计算强度的阈值，并返回一个有无环图
        
        #变为list，并且设置为df格式
        arc_ls = pd.DataFrame(to_strength_list(edges_strength_matrix.astype(np.float16)),columns=["from","to","strength"])
        #from和to的df格式并按大小排列
        arc_ls = arc_ls.sort_values(by="strength")
        #1.按照10个选择，但是白名单的列表应该不断的成长，而不是固定的数值
        #在每次讲融合DAG加入网络之后，先检验是否有这条边，如果有则跳过
    
        i = 0
        index = 0

        fuseDAG = DAG()
        while i <round_num*5:
            arc = (arc_ls.loc[i+index,"from"],arc_ls.loc[i+index,"to"])
            ##防止图的有环结构
            if fuseDAG.has_edge(arc[1],arc[0]):
                i=i+1
                continue
            fuseDAG.add_edge(arc[0],arc[1])
            
         
            i+=1
        #检查边是否有环，并且破环
        print("检查环")
        for root in fuseDAG.nodes():
            node_list=fuseDAG.get_ancestral_graph(root).nodes()
            for nd in node_list:
                if fuseDAG.has_edge(root,nd):
                    print(root,nd)
                    fuseDAG.remove_edge(root,nd)
        return  fuseDAG

In [76]:
DB_1 = pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
# DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/tenSiteDTDisc.csv").drop(columns=["Unnamed: 0","creatinine","patientunitstayid"],axis = 1)
DB_1=DB_1.sample(frac=1).reset_index(drop=True)
DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)
DB_1.fillna(1)
# 数据分割
# hos_id = [420,142,122,435,390,227,195,243,403,141]
hos_id = [420,142,122,435,390,227,144,140,396,141]##性能最好
# hos_id = [420,142,122,435,390,382,144,140,396,141]
# hos_id = [420,142,122,384,390,227,144,140,396,141]
# hos_id = [420,142,122,435,390,227,144,384,143,403]
# hos_id = [420,142,122,435,384,143,403,195,382,243]##性能最后后六个是外部验证
hos_id = np.sort(hos_id)
client_data = []
feature = DB_1.drop(columns = ["hospitalid"]).columns
global DB
DB = ro.r["data_tofactor"](DB_1)
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))
    
#设置黑名单
bl =[]
for i in feature:
    bl.append(["label",i])
    bl.append([i,"sex"])
    bl.append([i,"race"])
    bl.append([i,"bmi"])
    bl.append([i,"age"])
bl = pd.DataFrame(bl,columns=["from","to"])
# bl_r = ro.conversion.py2rpy(bl)
# globalenv['bl_r'] = bl_r
#计算样本量比重
D = 19388
D_W = []
for i in range(10):
    D_W.append(len(client_data[i])/D)
D_W = np.float16(D_W)

#将所有的离散状态记录下来，以字典的方式
state_names  ={
    "lable":[0,1],
    "age":[1,2,3,4,5,6,7,8],
    "sex":[1,2],
    "race":[1,2,3,4,5],
    "bmi":[1,2,3,4],
    "temperature":[1,2,3],
    "heartrate":[1,2,3],
    "respiration":[1,2,3],
    "SBP":[1,2,3,4,5],
    "DBP":[1,2,3,4,5],
    "paSystolic":[1,2,3],
    "paDiastolic":[1,2,3],
    "paMean":[1,2,3],
}
LAB_list = []
for i in range(1,61):
    LAB_list.append(("LAB"+str(i),[1,2,3]))
COM_list = []
for i in range(1,12):
    COM_list.append(("COM"+str(i),[0,1]))
PRO_list = []
for i in range(1,10):
    COM_list.append(("PRO"+str(i),[0,1]))
MED_list = []
for i in range(1,55):
    MED_list.append(("MED"+str(i),[0,1]))
state_names.update(LAB_list)
state_names.update(COM_list)
state_names.update(PRO_list)
state_names.update(MED_list)
BN_list = []
for i in range(1,11):
    BN_list.append("BN"+str(i))
    


In [77]:
all_score_init=[]
all_score_list=[]
fuse_dag_list = []
dag_struct_init_list = []
pandas2ri.activate()
scr_l=['fnml','bdla','mbde','bds','bic','loglik','bde', 'mbde', 'k2', 'qnml']
for scr in scr_l:
    if scr!="loglik":
        continue
    fuseDAG = DAG()
    
    globalenv['scr'] = scr

  
    # 结构训练
    train_round = 10
    round_num = 1
    client_num = 10
    clients = []

    for i in range(10):
    # 创建客户端，并且传入数据和结构学习模型
        clients.append(client(client_data[i],hos_id[i]))
    # 初始化服务器
    sever_1 = sever()
    #分别创建一个0/1矩阵和强度矩阵
    edges_strength_matrix_list =[]
    #记录边的频数
    global_edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    #记录边的强度
    edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    
    global_edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)

    score_list = []

    #先本地客户端建立初始模型,并且计算每个网络的初始性能
    print("计算每个网络的初始性能")
    for i in range(client_num):
        clients[i].bl_update(bl)
        clients[i].fit()
        dag_struct_init_list.append(clients[i].model)
        score_list.append(net_score(clients[i].model,DB,clients[i].cid))
    init_score = pd.DataFrame(score_list,index=BN_list,columns=["acc",'recall','auc','f1']).T
    all_score_init.append(init_score)
    #正式循环
    print("开始训练")
    while(round_num<train_round):
        #重置矩阵,设置评分
        global_edges_matrix.iloc[:,:] = 0
        edges_strength_matrix.iloc[:,:] = 0
        #get到每个客户端边强度的总和
        arc_strength_list = []
        arc_weight_list = []
        dag_score_list= []
        #开始聚集客户端边的信息

        for i in range(client_num):
            #将客户端最好的模型边信息转换为矩阵形式
            global_edges_matrix+=to_matrix(clients[i].model.edges(),feature)
            dag_score_list.append(clients[i].dag_score)
            #将矩阵转换为列表形式，并strength按列合并数据
        #在这里计算边的强度
        #进行分数的权重
        dag_score_list=np.array(dag_score_list)
        min_score = min(dag_score_list)
#         #看下输出结构的分数
#         for i in range(len(dag_score_list)):
#             dag_score_list[i]=dag_score_list[i]-min_score+1
#             for j in range(len(clients[i].arc_strength)):
#                 print(clients[i].arc_strength.sort_values(by="strength",ascending=False).iloc[j,:])

    
        #汇集带权重的边的信息
        for i in range(client_num):
            #边的强度*数据量权重*网络分数/遍的个数 
    #         print(get_label_str(clients[i]))

            global_edges_strength_matrix+=clients[i].edges_strength

            edges_strength_matrix+=clients[i].edges_strength*D_W[i]*dag_score_list[i]/len(clients[i].model.edges())

        print("转换")
        #将矩阵转换为，一个from->to + str 的表


        global_edges_strength_matrix_list = to_strength_list(global_edges_strength_matrix)
        print("输出未加权只是相加的强度")


        edges_strength_matrix_list.append(edges_strength_matrix)
    #     print("")
    #     for i in range(len(edges_strength_matrix_list)):
    #         print(edges_strength_matrix_list.iloc[i,:])

        fuseDAG = sever_1.Fd_caculate(edges_strength_matrix.round(4),round_num)
        
        
        print("转换完毕")
        #输出一下混合的结果
        for j in fuseDAG.edges:
            print(j[0],"->",j[1],";")



        #得到fuseDAG之后，观察整个网络评分的变化

        print("第",round_num+1,"次迭代")
        #跟新客户端

        #设置白名单
        wl = []
        for edge in fuseDAG.edges():
            wl.append([edge[0],edge[1]])

        wl = pd.DataFrame(wl,columns=["from","to"])
        print("传递客户端信息")
        for i in range(client_num):

            clients[i]. update_model(wl)

        round_num+=1
        
 

    
    
    



计算每个网络的初始性能
客户端 123 开始fit
客户端 141 开始fit
客户端 142 开始fit
客户端 143 开始fit
客户端 145 开始fit
客户端 228 开始fit
客户端 391 开始fit
客户端 397 开始fit
客户端 421 开始fit
客户端 436 开始fit
开始训练
转换
输出未加权只是相加的强度
检查环
转换完毕
LAB19 -> label ;
COM1 -> label ;
COM3 -> label ;
COM4 -> label ;
COM7 -> label ;
第 2 次迭代
传递客户端信息
客户端 122 开始update
(0.7738896366083445, 0.6872037914691943, 0.7477372340804619, 0.6331877729257642)
客户端 140 开始update
(0.8397435897435898, 0.8333333333333334, 0.8382352941176471, 0.782608695652174)
客户端 141 开始update
(0.7798594847775175, 0.8125, 0.7903769841269841, 0.6594202898550725)
客户端 142 开始update
(0.803680981595092, 0.7289156626506024, 0.7855104629042485, 0.7159763313609467)
客户端 144 开始update
(0.7965367965367965, 0.9032258064516129, 0.8303111280778774, 0.7044025157232705)
客户端 227 开始update
(0.8009708737864077, 0.45794392523364486, 0.6896277003217405, 0.5444444444444444)
客户端 390 开始update
(0.7936046511627907, 0.803030303030303, 0.7953830760434534, 0.7491166077738516)
客户端 396 开始update
(0.7839506172839507, 0.931034482

客户端 122 开始update
(0.7738896366083445, 0.6872037914691943, 0.7477372340804619, 0.6331877729257642)
客户端 140 开始update
(0.8397435897435898, 0.8333333333333334, 0.8382352941176471, 0.782608695652174)
客户端 141 开始update
(0.7845433255269321, 0.8392857142857143, 0.8021825396825396, 0.6714285714285715)
客户端 142 开始update
(0.803680981595092, 0.7289156626506024, 0.7855104629042485, 0.7159763313609467)
客户端 144 开始update
(0.7965367965367965, 0.9032258064516129, 0.8303111280778774, 0.7044025157232705)
客户端 227 开始update
(0.75, 0.8317757009345794, 0.7765435881722078, 0.6334519572953737)
客户端 390 开始update
(0.7936046511627907, 0.803030303030303, 0.7953830760434534, 0.7491166077738516)
客户端 396 开始update
(0.7839506172839507, 0.9310344827586207, 0.8164787798408487, 0.7552447552447552)
客户端 420 开始update
(0.8411214953271028, 0.7741116751269036, 0.8224490959904182, 0.7493857493857494)
客户端 435 开始update
(0.764102564102564, 0.8345864661654135, 0.7811064626546913, 0.7070063694267515)
转换
输出未加权只是相加的强度
检查环
转换完毕
LAB19 -> labe

In [83]:
score_list=[]
for i in range(client_num):
    score_list.append(net_score(clients[i].model,DB,clients[i].cid))
fuse_score_all  = pd.DataFrame(score_list,index=hos_id,columns=["acc",'recall','auc','f1']).T


In [84]:
score_list=[]
for i in range(10):
    score_list.append(net_score(fuseDAG,DB,hos_id[i]))
fuse_score  = pd.DataFrame(score_list,index=hos_id,columns=["acc",'recall','auc','f1']).T

In [85]:
fuse_score_all

,122,140,141,142,144,227,390,396,420,435
acc,0.773890,0.839744,0.786885,0.803681,0.796537,0.745146,0.793605,0.783951,0.839564,0.764103
recall,0.687204,0.833333,0.839286,0.728916,0.903226,0.841121,0.803030,0.931034,0.774112,0.834586
auc,0.747737,0.838235,0.803770,0.785510,0.830311,0.776298,0.795383,0.816479,0.821326,0.781106
f1,0.633188,0.782609,0.673835,0.715976,0.704403,0.631579,0.749117,0.755245,0.747549,0.707006


In [86]:
fuse_score

,122,140,141,142,144,227,390,396,420,435
acc,0.744280,0.724359,0.786885,0.760736,0.731602,0.747573,0.773256,0.709877,0.836449,0.733333
recall,0.744076,0.814815,0.839286,0.789157,0.854839,0.841121,0.871212,0.948276,0.774112,0.887218
auc,0.744218,0.745643,0.803770,0.767643,0.770615,0.777938,0.791738,0.762599,0.819078,0.770457
f1,0.623016,0.671756,0.673835,0.691293,0.630952,0.633803,0.746753,0.700637,0.743902,0.694118


In [87]:
all_score_init[0]

,BN1,BN2,BN3,BN4,BN5,BN6,BN7,BN8,BN9,BN10
acc,0.773890,0.839744,0.779859,0.803681,0.796537,0.800971,0.793605,0.783951,0.847352,0.764103
recall,0.687204,0.833333,0.812500,0.728916,0.903226,0.457944,0.803030,0.931034,0.738579,0.834586
auc,0.747737,0.838235,0.790377,0.785510,0.830311,0.689628,0.795383,0.816479,0.817042,0.781106
f1,0.633188,0.782609,0.659420,0.715976,0.704403,0.544444,0.749117,0.755245,0.748072,0.707006


In [74]:
print_dag(fuseDAG)

PRO3 -> label ;
LAB19 -> label ;
COM7 -> label ;
COM8 -> label ;
COM10 -> label ;
COM6 -> label ;
COM11 -> label ;
COM1 -> label ;
COM4 -> label ;
COM3 -> label ;


In [88]:
import plotly
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True) 
import plotly.graph_objs as go

In [ ]:
def plot_data(mth_list,idx):
    score_name = ["acc","recall","auc","f1"]
    plot_data = []
    for i,j in zip(mth_list,scr):
        trace = go.Scatter(x = [1,2,3,4,5,6,7,8,9,10],y =np.array(i.iloc[idx,:]),name = j,mode = "lines+markers")
        plot_data.append(trace)
    layout = go.Layout(
        xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
        yaxis_range=[0.6,0.91],
        
        yaxis=dict(
            tickmode="linear",
            tick0=0.8,
            dtick=0.05),
        
        title='不同方法在医院上对比_'+score_name[idx],
        titlefont={
        'size':25,
        'color':'#9ed900'
     })
    fig = go.Figure(data=plot_data,layout=layout)
    iplot(fig)

In [95]:
updated_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score_all.iloc[2,:]),
    name = 'updated_client',
    mode = "lines+markers")
fuse_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score.iloc[2,:]),
    name = 'fuse_dag',
    mode = "lines+markers")
init_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(all_score_init[0].iloc[2,:]),
    name = 'init_client',
    mode = "lines+markers")


layout = go.Layout(
    xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
    title='模型迭代前后对比'+"auc",
        titlefont={
        'size':25,
        'color':'#9ed900'
        })
data=[updated_auc,fuse_auc,init_auc]
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [98]:
updated_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score_all.iloc[3,:]),
    name = 'updated_client',
    mode = "lines+markers")
fuse_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score.iloc[3,:]),
    name = 'fuse_dag',
    mode = "lines+markers")
init_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(all_score_init[0].iloc[3,:]),
    name = 'init_client',
    mode = "lines+markers")


layout = go.Layout(
    xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
    title='模型迭代前后对比'+"f1",
        titlefont={
        'size':25,
        'color':'#9ed900'
        })
data=[updated_auc,fuse_auc,init_auc]
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [99]:
updated_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score_all.iloc[1,:]),
    name = 'updated_client',
    mode = "lines+markers")
fuse_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score.iloc[1,:]),
    name = 'fuse_dag',
    mode = "lines+markers")
init_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(all_score_init[0].iloc[1,:]),
    name = 'init_client',
    mode = "lines+markers")


layout = go.Layout(
    xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
    title='模型迭代前后对比'+"recall",
        titlefont={
        'size':25,
        'color':'#9ed900'
        })
data=[updated_auc,fuse_auc,init_auc]
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [101]:
updated_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score_all.iloc[0,:]),
    name = 'updated_client',
    mode = "lines+markers")
fuse_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(fuse_score.iloc[0,:]),
    name = 'fuse_dag',
    mode = "lines+markers")
init_auc = go.Scatter(
    x = [1,2,3,4,5,6,7,8,9,10],
    y =np.array(all_score_init[0].iloc[0,:]),
    name = 'init_client',
    mode = "lines+markers")


layout = go.Layout(
    xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
    title='模型迭代前后对比'+"acc",
        titlefont={
        'size':25,
        'color':'#9ed900'
        })
data=[updated_auc,fuse_auc,init_auc]
fig = go.Figure(data=data,layout=layout)
iplot(fig)